# OpenAI Tools

These output parsers extract tool calls from OpenAI’s function calling API responses. This means they are only usable with models that support function calling, and specifically the latest `tools` and `tool_choice` parameters. We recommend familiarizing yourself with [function calling](/docs/modules/model_io/chat/function_calling) before reading this guide.

There are a few different variants of output parsers:

- [`JsonOutputToolsParser`](https://api.js.langchain.com/classes/langchain_output_parsers.JsonOutputToolsParser.html): Returns the arguments of the function call as JSON
- [`JsonOutputKeyToolsParser`](https://api.js.langchain.com/classes/langchain_output_parsers.JsonOutputKeyToolsParser.html): Returns the value of specific key in the function call as JSON

In [7]:
import { ChatPromptTemplate } from '@langchain/core/prompts';
import { ChatOpenAI } from '@langchain/openai';

In [8]:
const properties = {
  setup: {
    type: "string",
    description: "The setup for the joke"
  },
  punchline: {
    type: "string",
    description: "The joke's punchline"
  }
};

const tool = {
  type: "function" as const,
  function: {
    name: "joke",
    description: "Joke to tell user.",
    parameters: {
      $schema: "http://json-schema.org/draft-07/schema#",
      title: "Joke",
      type: "object",
      properties,
      required: ["setup", "punchline"]
    },
  },
}

In [9]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const llm = new ChatOpenAI();

// Use `.bind` to attach the tool to the model
const llmWithTools = llm.bind({
  tools: [tool],
  // Optionally, we can pass the tool to the `tool_choice` parameter to
  // force the model to call the tool.
  tool_choice: tool,
});

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You are the funniest comedian, tell the user a joke about their topic."],
  ["human", "Topic: {topic}"]
])

Now we can use LCEL to pipe our prompt and LLM together.

In [10]:
const chain = prompt.pipe(llmWithTools);

In [12]:
const result = await chain.invoke({ topic: "Large Language Models" });

In [15]:
result.additional_kwargs;

{
  function_call: undefined,
  tool_calls: [
    {
      id: "call_vo9oYcHXKWzS6bJ4bK7Eghmz",
      type: "function",
      function: {
        name: "joke",
        arguments: "{\n" +
          '  "setup": "Why did the large language model go on a diet?",\n' +
          '  "punchline": "It wanted to reduce i'... 17 more characters
      }
    }
  ]
}

> #### Inspect the [LangSmith trace](https://smith.langchain.com/public/f2f34c8d-8193-40cb-b3ef-f186fb4de73e/r) from the call above

## `JsonOutputToolsParser`

In [16]:
import { JsonOutputToolsParser } from "langchain/output_parsers";

const outputParser = new JsonOutputToolsParser();

In [17]:
const chain = prompt.pipe(llmWithTools).pipe(outputParser);

In [18]:
await chain.invoke({ topic: "Large Language Models" });

[
  {
    type: "joke",
    args: {
      setup: "Why did the large language model go to therapy?",
      punchline: "It had too many layers!"
    }
  }
]

> #### Inspect the [LangSmith trace](https://smith.langchain.com/public/61ce7b9f-d462-499e-be65-8a165d2b47a7/r) with the `JsonOutputToolsParser`

## `JsonOutputKeyToolsParser`

This merely extracts a single key from the returned response. This is useful for when you are passing in a single tool and just want it’s arguments.

In [19]:
import { JsonOutputKeyToolsParser } from "langchain/output_parsers";

const outputParser = new JsonOutputKeyToolsParser({ keyName: "joke" });

In [20]:
const chain = prompt.pipe(llmWithTools).pipe(outputParser);

In [21]:
await chain.invoke({ topic: "Large Language Models" })

[
  {
    setup: "Why did the large language model go to therapy?",
    punchline: "It had too many layers!"
  }
]

> #### Inspect the [LangSmith trace](https://smith.langchain.com/public/2c9c93d2-d789-4e45-9f9f-e942eace8aed/r) with the `JsonOutputKeyToolsParser`

Some LLMs have support for calling multiple tools in a single response. Because of this, the result of invoking `JsonOutputKeyToolsParser` is always an array. If you would only like a single result to be returned, you can specify `returnSingle` in the constructor.

In [22]:
const outputParserSingle = new JsonOutputKeyToolsParser({
  keyName: "joke",
  returnSingle: true,
});

In [23]:
const chain = prompt.pipe(llmWithTools);

In [24]:
const response = await chain.invoke({ topic: "Large Language Models" });

In [26]:
await outputParserSingle.invoke(response)

{
  setup: "Why did the large language model go on a diet?",
  punchline: "It wanted to shed some excess bytes!"
}

> #### See the [LangSmith trace](https://smith.langchain.com/public/c05e0409-8085-487d-aee2-2d42b64b9f6d/r) from this output parser.